In [ ]:
import cv2
import numpy as np
import pandas as pd
import os
import sys

def detect_cross_vertical(x, line_x, buffer=2):
    """
    Detect if an object crosses the vertical line.
    """  

    return -3 <= (line_x - x) <= 1.5

def process_video_buzzy_beetles(video_path, output_path, crop_coords):
    """
    Process the video to count Buzzy Beetles (blue shells) crossing a vertical line
    in a specific region and save the output video with visualization.
    """
    count_beetles = 0
    x_start, y_start, x_end, y_end = crop_coords

    # Open the video file
    cap = cv2.VideoCapture(video_path)


    # Check if the video was loaded successfully
    if not cap.isOpened():
        print(f"Error: Unable to open video file {video_path}. Please check the path.")
        return 0  # Return 0 if video can't be processed

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    line_x = frame_width // 2 - 140  # Vertical line in the middle of the frame

    # Define the codec and create VideoWriter object to save the output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    # Store beetles that have crossed the line (by unique id)
    crossed_beetles = set()

    frame_num = 0
    beetles_last_position = {}


    while True:
        frame_num += 1
        grabbed, frame = cap.read()

        if not grabbed:  # If no frame is captured, end processing
            break

        # Convert the frame to HSV color space
        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Crop the region of interest
        roi = hsv_frame[y_start:y_end, x_start:x_end]

        # Define HSV range for blue shells (adjust these values if needed)
        lower_blue = np.array([100, 150, 50])  # Lower HSV bounds for blue
        upper_blue = np.array([140, 255, 255])  # Upper HSV bounds for blue

        # Create a mask for blue color in the ROI
        mask = cv2.inRange(roi, lower_blue, upper_blue)

        # Find contours of blue objects in the ROI
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            if 20 <= w <= 80 and 20 <= h <= 80:  # Ignore small detections (noise)

                center_x = x + w // 2
                center_y = y + h // 2

                # Adjust coordinates relative to the full frame
                global_center_x = x_start + center_x
                global_center_y = y_start + center_y

                # Track beetles by their center positions and bounding box size
                beetle_id = (global_center_x, global_center_y)

                # Check if the beetle has crossed the line
                if detect_cross_vertical(global_center_x, line_x) and beetle_id not in crossed_beetles:
                    # Mark beetle as crossed once
                    crossed_beetles.add(beetle_id)  # Store the beetle's id
                    count_beetles += 1
                    #print(f"Beetle crossed at: {global_center_x}, {global_center_y}")

                # Draw bounding box and center point on the full frame
                cv2.rectangle(frame, (x_start + x, y_start + y), (x_start + x + w, y_start + y + h), (255, 0, 0), 2)
                cv2.circle(frame, (global_center_x, global_center_y), 5, (0, 0, 255), -1)

        # Draw the vertical line on the full frame
        cv2.line(frame, (line_x, 0), (line_x, frame_height), (0, 255, 0), 2)

        # Overlay the count on the frame
        cv2.putText(frame, f"Buzzy Beetles: {count_beetles}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Write the frame to the output video
        out.write(frame)

    # Release video objects
    cap.release()
    out.release()
    #print(count_beetles)
    return count_beetles


def calculate_mae(predictions, expected_counts):

    mae = sum(abs(pred - expected) for pred, expected in zip(predictions, expected_counts)) / len(predictions)
    return mae


def process_multiple_videos(folder_path, crop_coords):
    # Load the metadata file
    metadata_path = os.path.join(folder_path, "buzzy_beetle_count.csv")
    if not os.path.exists(metadata_path):
        print(f"Error: Metadata file {metadata_path} not found.")
        return

    df = pd.read_csv(metadata_path)

    predictions = []
    expected_counts = df['count'].tolist()

    for idx, row in df.iterrows():
        video_path = os.path.join(folder_path, row['video'])
        output_path = f"output_{os.path.basename(video_path).replace('.mp4', '')}_debug.mp4"
        # print(f"Processing {video_path}...")  # Uncomment to enable debug logging

        count_beetles = process_video_buzzy_beetles(video_path, output_path, crop_coords)
        predictions.append(count_beetles)

    # Calculate MAE
    mae = calculate_mae(predictions, expected_counts)
    print(mae)

if __name__ == "__main__":

    folder_path = "data"
    crop_coords = (400, 100, 900, 620)  # x_start, y_start, x_end, y_end

    process_multiple_videos(folder_path, crop_coords)
